# DATA612 - Assignment # 1
# BY: Sang Yoon (Andy) Hwang
# DATE: 2019-06-11

In [69]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split

# Description of recommender system
This system recommends movies to people based on movie ratings (1 to 5).

In [70]:
# Just create dataset with this format --> user, movie_title, rating and then use pivot/melt to create matrix
data = {'user':['Tom','Sally','Vincio','Andy','Mike'], 'Titanic':[5,4,2,5, None],'Batman':[5,5,2,4,3], 'Superman':[4,5,3,None,4], 'Spiderman':[None, 4,1,3,4], 'Avengers':[4,5,3, None, None] }
df = pd.DataFrame(data)
df

,user,Titanic,Batman,Superman,Spiderman,Avengers
0,Tom,5.0,5,4.0,NaN,4.0
1,Sally,4.0,5,5.0,4.0,5.0
2,Vincio,2.0,2,3.0,1.0,3.0
3,Andy,5.0,4,NaN,3.0,NaN
4,Mike,NaN,3,4.0,4.0,NaN


In [85]:
# long-form dataframe
df2 = pd.melt(df, 
               id_vars='user', 
               var_name='movie', 
               value_name='rating')
df2

,user,movie,rating
0,Tom,Titanic,5.0
1,Sally,Titanic,4.0
2,Vincio,Titanic,2.0
3,Andy,Titanic,5.0
4,Mike,Titanic,NaN
5,Tom,Batman,5.0
6,Sally,Batman,5.0
7,Vincio,Batman,2.0
8,Andy,Batman,4.0
9,Mike,Batman,3.0


# Split 80/20 (train/test) for df2 (converted to long-form), by removing points in row and column 

In [72]:
train2, test2 = train_test_split(df2, test_size = 0.2, random_state = 42)

In [73]:
train2 = pd.pivot_table(train2, index = 'user', columns = 'movie', values = 'rating' ).reset_index()
train2

movie,user,Avengers,Batman,Spiderman,Superman,Titanic
0,Andy,NaN,NaN,3.0,NaN,5.0
1,Mike,NaN,3.0,4.0,4.0,NaN
2,Sally,5.0,5.0,NaN,NaN,4.0
3,Tom,4.0,5.0,NaN,4.0,NaN
4,Vincio,3.0,2.0,1.0,3.0,2.0


In [74]:
test2 = pd.pivot_table(test2, index = 'user', columns = 'movie', values = 'rating' ).reset_index()
test2

movie,user,Batman,Spiderman,Superman,Titanic
0,Andy,4.0,NaN,NaN,NaN
1,Sally,NaN,4.0,5.0,NaN
2,Tom,NaN,NaN,NaN,5.0


# Raw average

In [75]:
raw_avg2 = train2.sum()[1:].sum() / train2.iloc[:,1:].notnull().sum().sum()
raw_avg2

3.5625

# RMSE for test set - df2

In [76]:
error_test2 = test2.iloc[:,1:] - raw_avg2
sq_error_test2 = error_test2 ** 2 
MSE_test2 = sq_error_test2.sum().sum() / test2.iloc[:,1:].notnull().sum().sum()
RMSE_test2 = np.sqrt(MSE_test2)
RMSE_test2

1.0625

# RMSE for training set - df2

In [77]:
error_train2 = train2.iloc[:,1:] - raw_avg2
sq_error_train2 = error_train2 ** 2 
MSE_train2 = sq_error_train2.sum().sum() / train2.iloc[:,1:].notnull().sum().sum()
RMSE_train2 = np.sqrt(MSE_train2)
RMSE_train2

1.1709371246996996

# Bias Values

In [78]:
#bias_user using df2
sum_item_user2 = train2.sum(axis=1)
n_item_user2 = train2.iloc[:,1:].notnull().sum(axis=1)

bias_user2 = (sum_item_user2 / n_item_user2) - raw_avg2
bias_user2.index = list(train2.user)
bias_user2

Andy      0.437500
Mike      0.104167
Sally     1.104167
Tom       0.770833
Vincio   -1.362500
dtype: float64

In [79]:
#bias_movie using df2
sum_item_movie2 = train2.iloc[:,1:].sum(axis=0)
n_item_movie2 = train2.iloc[:,1:].notnull().sum(axis=0)

bias_movie2 = (sum_item_movie2 / n_item_movie2) - raw_avg2
bias_movie2

movie
Avengers     0.437500
Batman       0.187500
Spiderman   -0.895833
Superman     0.104167
Titanic      0.104167
dtype: float64

# baseline predictor

In [80]:
#using df2 - calculate baseline RMSE for testset
bias_movie_df2 = pd.DataFrame(bias_movie2).T
base_line_train2 = pd.DataFrame(columns = bias_movie2.index)
for i in train2['user']:
    base_line_train2 = pd.concat([bias_movie_df2 + bias_user2[i] + raw_avg2, base_line_train2])

In [94]:
base_line_train2.index = train2.index
base_line_train2

movie,Avengers,Batman,Spiderman,Superman,Titanic
0,2.637500,2.387500,1.304167,2.304167,2.304167
1,4.770833,4.520833,3.437500,4.437500,4.437500
2,5.104167,4.854167,3.770833,4.770833,4.770833
3,4.104167,3.854167,2.770833,3.770833,3.770833
4,4.437500,4.187500,3.104167,4.104167,4.104167


In [95]:
error_test2_baseline = test2 - base_line_train2
sq_error_test2_baseline = error_test2_baseline ** 2
MSE_test2_baseline = sq_error_test2_baseline.sum().sum() / test2.iloc[:,1:].notnull().sum().sum()
RMSE_test2_baseline = np.sqrt(MSE_test2_baseline)
RMSE_test2_baseline

0.9062954969422378

In [96]:
error_train2_baseline = train2 - base_line_train2
sq_error_train2_baseline = error_train2_baseline ** 2
MSE_train2_baseline = sq_error_train2_baseline.sum().sum() / train2.iloc[:,1:].notnull().sum().sum()
RMSE_train2_baseline = np.sqrt(MSE_train2_baseline)
RMSE_train2_baseline

1.4118361271762385

# Summary : RMSE comparison

In [67]:
print('RMSE - raw average: training', RMSE_train2)
print('RMSE - baseline predictor: training', RMSE_train2_baseline)
print('RMSE - raw average: test', RMSE_test2)
print('RMSE - baseline predictor: test', RMSE_test2_baseline)

RMSE - raw average: training 1.1709371246996996
RMSE - baseline predictor: training 1.4118361271762385
RMSE - raw average: test 1.0625
RMSE - baseline predictor: test 0.9062954969422378


Interestingly, RMSE improved using baseline predictors for both training and test sets. However, we should keep in mind that this might be due to a small data set with very limited information. The testing set contains only 4 ratings with only 3 out of 5 users. Data set was created manually so calculating biases, such as harsh critic or generally favorable performance, might not have done effectively.